In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = '<your_kaggle_username>'
os.environ['KAGGLE_KEY'] = '<your_kaggle_key>'

In [ ]:
!pip install keras-nlp
!pip install keras>=3.0.0

In [ ]:
import keras
keras.backend.set_backend('jax')

In [ ]:
import os
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '1.0'

In [ ]:
import json
with open('databricks-dolly-15k.json', 'r') as f:
    data = json.load(f)
processed_data = []
for item in data:
    if 'context' not in item:
        continue
    processed_data.append({
        'instruction': item['instruction'],
        'response': item['context']
    })
top_1000_data = processed_data[:1000]

In [ ]:
import keras_nlp
model = keras_nlp.models.GemmaCausalLM.from_preset('gemma-2b')

In [ ]:
template = {'instruction': 'What should I do on a trip to Europe?', 'response': ''}
sampler = keras_nlp.samplers.TopKSampler(k=5)
model.compile(sampler=sampler)
results = model.generate(template)

In [ ]:
template = {'instruction': 'Explain the process of photosynthesis in a way that a child could understand.', 'response': ''}
results = model.generate(template)

In [ ]:
model.enable_lora(rank=4)
model.summary()

In [ ]:
from keras.optimizers import AdamW
optimizer = AdamW(learning_rate=5e-4, weight_decay=0.01)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

In [ ]:
model.fit(top_1000_data, epochs=1, batch_size=1)

In [ ]:
template = {'instruction': 'What should I do on a trip to Europe?', 'response': ''}
results = model.generate(template)

In [ ]:
template = {'instruction': 'Explain the process of photosynthesis in a way that a child could understand.', 'response': ''}
results = model.generate(template)